# **Meerkat Call Detection Framework**

This notebook allows you to go through the full process of training a CNN on meerkat calls, running the CNN on new files to extract potential calls, and then evaluating the effectiveness using ROC curves. You can go through all steps or perform a subset of them, but first choose which files you will be using for training and testing.

## Set parameters
Here, you set up the parameters needed for the training and testing. Run the cell below before you run the rest of the notebook!

**Directories**

All directories need to be accessible from your Google Drive.

*audio_dir*: This should contain 3 subfolders. 'calls' and 'noise' are folders where the call and noise snippets used for training are located. 'long_recordings' is where full audio files are

*groundtruth_dir*: This is the directory containing csv files of labeled data

*model_dir*: This directory contains fitted models

*output_dir*: This is where the output (.pckl and .csv files) will be stored

*code_dir*: Directory where code is stored

**Files**

*model_name*: file name for the output model, or for the model to load (in the case of using a pre-trained model)

**Training parameters**

*epochs*: number of epochs to train for

*batch_size*: batch size of data to use

*steps_per_epoch*: how many training steps per epoch

**Prediction parameters**

*audio_file*: name of audio file to run prediction on

*t_start*: start time for predictions within audio file (sec)

*t_end*: end time for predictions within audio file (sec)

**Other parameters (probably don't need to change)**

*samprate*: sample rate of audio file, should be 8000 for current code (code will likely not work with other sample rates!)

*chunk_size*: number of seconds to read in each time wav is accessed directly

*chunk_pad*: pad chunks of wavs on each end to avoid any issues - 1 sec is fine

In [ ]:
#TODO: Fix evaluation, need to match up file names due to 8000_LEFT extension

#PARAMETERS - Modify parameters before running to change settings!
#-------------------------------------------------------------------------------

#---------TO CHANGE--------:

#General
use_pretrained_model = False #whether to load a pre-trained model (either to go to evaluation or to continue training with new data)
train_model = True #whether or not to train model or skip straight to evaluation
run_model_on_file = False
run_model_on_folder = True
run_model_on_specified_round = False

specified_round = 2
specified_model_name = None #defaults to None. use only if use_pretrained_model is True or if you want to specify the name of the model directly
run_only_where_ground_truth_available = True #set to True to run only on parts of files where groundtruth labels are available
evaluate_detections = True
focal = False #whether to use focal recordings (or collar data if false)
focal_megan = False

foc_megan_dir = '/media/arianasp/Elements/Sound Files/Ari Examined files/A. Clean'
ground_truth_path_test_megan = '/home/arianasp/meerkat_detector/labels_megan/focal_labels_test.csv'

#Model fitting options (use only if train_model is True)
epochs = 20 #Number of epochs to train for
augment = False #whether to augment by overlaying noise (at different levels) on calls
conv_dimension = 2

#name of audio file (use only if run_model_on_file is True)
#audio_file_to_predict = 'HM_PET_R11_20170903-20170908_file_2_(2017_09_03-05_44_59)_ASWMUX221163_SS.wav'
audio_file_to_predict = 'OCS1451_VJXM100_L_1_6FEB2017_LEFT_8000.wav'
#audio_file_to_predict = 'HM_VLF206_SOUNDFOC_20170903_LABEL_RY_downsamp.wav'

#Name of audio folder for prediction (use only if run_model_on_folder is True)
audio_folder = '/home/arianasp/meerkat_detector/data/full_recordings'

#Probability of selecting each call type for training (call types given below). If None, choose calls with probability equal to their occurrence in the training data
call_probs = None

#---------TO LEAVE ALONE (PROBABLY)--------:

#Main directory
base_dir = '/home/arianasp/meerkat_detector' #base directory

#Subdirectories
audio_dir = base_dir + '/data/full_recordings'
if focal_megan:
    audio_dir = foc_megan_dir

ground_truth_dir = base_dir + '/ground_truth'
model_dir = base_dir + '/models'
output_dir = base_dir + '/predictions'
code_dir = base_dir + '/dev'
if(focal):
    clips_dir = base_dir + '/clips_foc'
elif(focal_megan):
    clips_dir = base_dir +'/clips_foc_megan_all'
else:
    clips_dir = base_dir + '/clips'
eval_dir = base_dir + '/eval'

#Training parameters
batch_size = 100
steps_per_epoch = 1000 #1000
call_types = ['cc','sn','ld','mov','agg','alarm','soc','hyb','unk','oth']

#Evaluation parameters
boundary_thresh = 0.6
n_points = 30
pckl_paths = ['/home/arianasp/meerkat_detector/predictions/OCS1451_VJXM100_L_1_6FEB2017_LEFT_8000_label_cnn_10epoch__focalmeg__orig_proportional_noaug_2dconv_20190306_1-1094.7.pckl'] #NOTE: One can specify pckl_paths, but this is only useful when running evluation ALONE, otherwise anything specified here gets cleared

#Other parameters
samprate = 8000 
chunk_size = 60 
chunk_pad = 1 
ml_plan_file = base_dir + '/docs/' + 'audio_labeling_plan_filenames.csv'
learn_rate = 2e-5

#SETUP
#-------------------------------------------------------------------------------

#import libraries
import sys
import os
import wave
import time
import glob
import audioread

#Set path
sys.path.append(code_dir)

#Import call detector library
from meerkat_call_detector_library import *

#NEW MODEL CONSTRUCTION DEFINITIONS (TO MOVE LATER)
#inputs: originally spectrogram or output of upper layer
#filters: number of filters to use (arbitrary)
#n_convs: number of consecutive convolutions to do
#output will be time_dim(inputs)/2 x n_filters


#Set up file names and paths
aug_str = 'noaug'
if(augment):
    aug_str = 'aug'
    
dim_str = '_1dconv_'
if(conv_dimension==2):
    dim_str = '_2dconv_'
    
focal_str = '_collar_'
if(focal):
    focal_str = '_focal_'
elif(focal_megan):
    focal_str = '_focalmeg_'

if(call_probs is not None):
    call_probs_str = ''.join([str(s) + '_' for s in call_probs])
else:
    call_probs_str = 'proportional_'
    
pretrain_str = '_orig_'
if(use_pretrained_model):
    pretrain_str = '_pretrained_'

#if model name was specified, use specified name here. otherwise construct name based on parameters.
if(not(train_model)):
    model_name = specified_model_name
else:
    model_name = 'cnn_' + str(epochs) + 'epoch_' +  focal_str + pretrain_str + call_probs_str + aug_str + dim_str + time.strftime('%Y%m%d') + '.h5'
model_path = model_dir + '/' + model_name

#create threshold range for evaluation
thresh_range = np.linspace(boundary_thresh+.0001,.9,10)

for i in range(2,n_points-9):
    thresh_range = np.append(thresh_range,thresh_range[len(thresh_range)-1]+10**(-i)*9)
    
#TRAIN OR LOAD MODEL
#-------------------------------------------------------------------------------
        
if(use_pretrained_model):

    print("-------- Loading pretrained model --------")
    print('Model name: ' + specified_model_name)
  
    #Load pre-trained model
    model = load_model(model_dir + '/' + specified_model_name)

if(train_model):
  
    print("-------- Training model --------")
    print('Model name: ' + model_name)
    print("Clips directory: " + clips_dir)
    print('CNN Dimension: ' + str(conv_dimension))
    print('Start time:')
    print(datetime.datetime.now())

    if(not(use_pretrained_model)):
        print('Creating new model')
        #Construct model
        if(conv_dimension==1):
            model = construct_unet_model(lr=learn_rate)
        else:
            model = construct_unet_model_2d(lr=learn_rate)
        model.summary()

    #Fit model
    model.fit_generator(data_generator(clips_dir = clips_dir,batch_size = batch_size, cnn_dim=conv_dimension), epochs=epochs, use_multiprocessing=True, workers=16, steps_per_epoch=steps_per_epoch)

    print('End time:')
    print(datetime.datetime.now())

    #Save fitted model
    print('Saving model as: ' + model_name)
    model.save(filepath=model_dir + '/' + model_name) 

print("-------- Done with training or loading model step --------")


#RUN MODEL TO DETECT CALLS
#-------------------------------------------------------------------------------
#Extract probable calls from wav recording
                    
if(run_model_on_folder):
    
    print("-------- Running model on folder --------")
    print('Folder path = ' + audio_folder)
    
    #get all audio files in that folder (or subfolders of it, recursively)
    audio_files = glob.glob(audio_folder + '/**/' + '*.wav',recursive=True)
    
    #print number of files found
    print('Found ' + str(len(audio_files)) + ' audio files, running model on all of them')
    
if(run_model_on_specified_round):
    
    print("-------- Running model on specified round --------")
    print('Round = ' + str(specified_round))

    ml_plan = pandas.read_csv(ml_plan_file)
    files_to_run = ml_plan[(ml_plan['Round (0-1)'] == str(specified_round)) | (ml_plan['Round (1-2)'] == str(specified_round)) | (ml_plan['Round (2+)'] == str(specified_round))]
    files_to_run = files_to_run['Audio filename'].tolist()

    audio_files = list()

    for f_idx in range(len(files_to_run)):
        curr_file = glob.glob(base_dir + '/data/raw_data' + '/**/' + files_to_run[f_idx], recursive=True)[0]
        audio_files.append(curr_file)

    #print number of files found
    print('Found ' + str(len(audio_files)) + ' audio files, running model on all of them')

if(run_model_on_folder or run_model_on_specified_round):
    
    for i in range(len(audio_files)):
        
        audio_file = audio_files[i]
        wav_path = audio_file
        
        print('Running predictions on file: ')
        print(audio_file)
        
        #aud = wave.open(audio_file,'rb')
        
        #time bounds for extraction
        if(run_only_where_ground_truth_available):
            labels = get_ground_truth_labels(wav_name = os.path.basename(audio_file), ground_truth_dir = ground_truth_dir)
            if(labels is None):
                print('No ground truth data found - skipping this file')
                continue
            else:
                [t_start, t_end] = get_start_end_time_labels(labels)
        else:
            t_start = 1
            with audioread.audio_open(wav_path) as f:
                t_end = f.duration - 1.5
        
        #start at least 1 sec in to avoid problems of wrong input size in next step
        if(t_start < 1):
            t_start = 1
        
        #Store parameters in extraction_params object
        audio_name = os.path.basename(audio_file)
        pckl_path = output_dir + '/' + audio_name[0:(len(audio_name)-4)] + "_label_" + model_name[0:(len(model_name)-3)] + '_' + str(t_start) + '-' + str(t_end) + ".pckl"
        
        #Append to list of created pckl paths
        pckl_paths.append(pckl_path)
        
        #if path to extraction results already exists, do not run. otherwise run.
        if(not(os.path.exists(pckl_path))):
            extraction_params = CallExtractionParams(model_path = model_path, wav_path = wav_path, pckl_path=pckl_path, samprate = samprate, t_start = t_start, t_end = t_end)
            print(extraction_params)
            
            #if SOUNDFOC is in filename, this indicates a different type of sound file - don't run!
            if(re.search('SOUNDFOC',audio_file)==None):
                extract_scores(model, extraction_params)

#Run model on a specific file
if(run_model_on_file):
    
    print('--------Running model on specific file---------')

    #create paths to prediction files (wav and pckl)
    wav_path = audio_dir + '/' + audio_file_to_predict
    
    print(wav_path)
    
    #tibase_dir = '/home/arianasp/meerkat_detector'me bounds for extraction
    if(run_only_where_ground_truth_available): #TODO: add option to find labeled portion and run only for this
        labels = get_ground_truth_labels(wav_name = audio_file_to_predict, ground_truth_dir = ground_truth_dir)
        if(labels is None):
            print('No ground truth data found - set run_only_where_ground_truth_available to False to run on this file')
            t_start = None
        else:
            [t_start, t_end] = get_start_end_time_labels(labels)
            t_end = t_end - 1
        #start at least 1 sec in to avoid problems with wrong matrix size in next step
        if t_start < 1:
            t_start = 1
    else:
        t_start = 1
        with audioread.audio_open(wav_path) as f:
            t_end = f.duration - 1.5
        
    if(t_start is not None):
    
        #create path to output file
        pckl_path = output_dir + '/' + audio_file_to_predict[0:(len(audio_file_to_predict)-4)] + "_label_" + model_name[0:(len(model_name)-3)] + '_' + str(t_start) + '-' + str(t_end) + ".pckl"
    
        pckl_paths = [pckl_path]
        extraction_params = CallExtractionParams(model_path = model_path, wav_path = wav_path, pckl_path = pckl_path, samprate = samprate, t_start = t_start, t_end = t_end)
        print(extraction_params.t_start)
        print(extraction_params.t_end)
        extract_scores(model, extraction_params)
    
#EVALUATE DETECTIONS
#-------------------------------------------------------------------------------
if(evaluate_detections & (pckl_paths is not None)):
    
    #for file_idx in range(len(pckl_files)):
    for file_idx in range(len(pckl_paths)):

        pckl_path = pckl_paths[file_idx]

        if(focal_megan):
            run_evaluation(pckl_path = pckl_path,thresh_range=thresh_range,save_dir =eval_dir,ground_truth_dir = None,call_types = call_types, verbose = False, ground_truth_path = ground_truth_path_test_megan, foc_megan = True)
        else:
            run_evaluation(pckl_path = pckl_path,thresh_range=thresh_range,save_dir =eval_dir,ground_truth_dir = ground_truth_dir,call_types = call_types, verbose = False)

-------- Training model --------
Model name: cnn_20epoch__collar__orig_proportional_noaug_2dconv_20190531.h5
Clips directory: /home/arianasp/meerkat_detector/clips
CNN Dimension: 2
Start time:
2019-05-31 13:19:26.278611
Creating new model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 512, 128, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
activation_26 (Activation)      (None, 512, 128, 32) 0           conv2d_32[0][0]                  
____________________________________________________________________

Epoch 1/20
1000/1000 [==============================] - 422s 422ms/step - loss: 0.1924
Epoch 2/20
1000/1000 [==============================] - 421s 421ms/step - loss: 0.1031
Epoch 3/20
1000/1000 [==============================] - 423s 423ms/step - loss: 0.1014
Epoch 4/20
1000/1000 [==============================] - 424s 424ms/step - loss: 0.0802
Epoch 5/20
1000/1000 [==============================] - 423s 423ms/step - loss: 0.0794
Epoch 6/20
1000/1000 [==============================] - 423s 423ms/step - loss: 0.0783
Epoch 7/20
1000/1000 [==============================] - 426s 426ms/step - loss: 0.0760
Epoch 8/20
1000/1000 [==============================] - 424s 424ms/step - loss: 0.0665
Epoch 9/20
1000/1000 [==============================] - 423s 423ms/step - loss: 0.0630
Epoch 10/20
1000/1000 [==============================] - 426s 426ms/step - loss: 0.0566
Epoch 11/20
1000/1000 [==============================] - 424s 424ms/step - loss: 0.0624
Epoch 12/20
1000/1000 [==================

End time:
2019-05-31 15:53:30.172707

-------Generating scores and extracting calls-------
-------Saving output-------
Done
Running predictions on file: 
/home/arianasp/meerkat_detector/data/full_recordings/HM_HRT_R09_AUDIO_file_6_(2017_08_25-06_44_59)_ASWMUX221110_label.wav
/home/arianasp/meerkat_detector/ground_truth/HM_HRT_R09_AUDIO_file_6_(2017_08_25-06_44_59)_ASWMUX221110_label.csv
-------Running CNN model on new data-------
model_path: /home/arianasp/meerkat_detector/models/cnn_20epoch__collar__orig_proportional_noaug_2dconv_20190531.h5
wav_path: /home/arianasp/meerkat_detector/data/full_recordings/HM_HRT_R09_AUDIO_file_6_(2017_08_25-06_44_59)_ASWMUX221110_label.wav
samprate: 8000
t_start: 3600.0
t_end: 7325.0

--------Generating predictions-------
Start time:
2019-05-31 15:53:30.341360
